#Crime Category Prediction Using Machine Learning

- Spark ML's Machine Learning Models were used to predict the crime "Category" from the input feature vector consisted of ["DayOfWeek", "Month", "Date", "Time", "PdDistrict", "X", "Y"]
- Random Forest Classifier Model and the Multi-layer Perception Classifier Model were used

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext

In [3]:
sqlContext = SQLContext(sc)

In [4]:
dataPath = '/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv'
crimeDataSchema = StructType([StructField("IncidntNum", LongType(), True),
                              StructField("Category", StringType(), True),
                              StructField("Descript", StringType(), True),
                              StructField("DayOfWeek", StringType(), True),
                              StructField("Date", StringType(), True),
                              StructField("Time", StringType(), True),
                              StructField("PdDistrict", StringType(), True),
                              StructField("Resolution", StringType(), True),
                              StructField("Address", StringType(), True),
                              StructField("X", DoubleType(), True),
                              StructField("Y", DoubleType(), True),
                              StructField("Location", StringType(), True),
                              StructField("PdId", LongType(), True)])

crimeDF = (sqlContext.read
           .format('csv')
           .option('delimiter', ',')
           .option('header', 'true')
           .load(dataPath, schema=crimeDataSchema))

In [5]:
crimeDF = crimeDF.filter("PdDistrict is not NULL")

In [6]:
cats = crimeDF.select("Category").distinct().collect()

In [7]:
Category_idx = {}

for i in range(39):
    Category_idx.update({cats[i][0]:i})

In [8]:
day_idx = {"Monday":0,
           "Tuesday":1,
           "Wednesday":2,
           "Thursday":3,
           "Friday":4,
           "Saturday":5,
           "Sunday":6
          }

district_idx = {"MISSION":0,
                "BAYVIEW":1,
                "CENTRAL":2,
                "TARAVAL":3,
                "TENDERLOIN":4,
                "INGLESIDE":5,
                "PARK":6,
                "SOUTHERN":7,
                "RICHMOND":8,
                "NORTHERN":9
               }

For the dataset,
- "Date" column is parsed and decomposed into Month and Date
- "Time" column is parsed to extract out the hour from the time
- "PdDistrict" is encoded with district_idx
- "DayOfWeek" is encoded following day_idx
- "Category" is encoded with Category_idx

In [10]:
import datetime

# def parseDate(dateStr):
#     tokens = dateStr.split("/")
    
#     month = int(tokens[0])
#     date = int(tokens[1])
#     year = int(tokens[2])
#     return datetime.date(year, month, date)
  
def parseMonth(dateStr):
    tokens = dateStr.split("/")    
    return int(tokens[0])

def parseDate(dateStr):
    tokens = dateStr.split("/")    
    return int(tokens[1])

def parseTime(timeStr):
    tokens = timeStr.split(":")
    hour = int(tokens[0])
    minute = int(tokens[1])
    return hour

def parseDay(dayStr):
    return day_idx[dayStr]

def parseDist(distStr):
    return district_idx[distStr]

def parseCategory(catStr):
    return Category_idx[catStr]

from pyspark.sql.functions import udf

crimeDF = (crimeDF.withColumn("Date_tmp", udf(parseDate, IntegerType())(crimeDF.Date))
           .withColumn("Month_tmp", udf(parseMonth, IntegerType())(crimeDF.Date))
           .withColumn("Time_tmp", udf(parseTime, IntegerType())(crimeDF.Time))
           .withColumn("Day_tmp", udf(parseDay, IntegerType())(crimeDF.DayOfWeek))
           .withColumn("District_tmp", udf(parseDist, IntegerType())(crimeDF.PdDistrict))
           .withColumn("Cats_tmp", udf(parseCategory, IntegerType())(crimeDF.Category))
           .drop("Time")
           .drop("Date")
           .drop("DayOfWeek")
           .drop("PdDistrict")
           .drop("Category")
           .withColumnRenamed("Date_tmp", "Date")
           .withColumnRenamed("Month_tmp", "Month")
           .withColumnRenamed("Time_tmp", "Time")
           .withColumnRenamed("Day_tmp", "DayOfWeek")
           .withColumnRenamed("District_tmp", "PdDistrict")
           .withColumnRenamed("Cats_tmp", "Category")
          ).cache()

In [11]:
display(crimeDF)

IncidntNum,Descript,Resolution,Address,X,Y,Location,PdId,Date,Month,Time,DayOfWeek,PdDistrict,Category
150060275,LOST PROPERTY,NONE,18TH ST / VALENCIA ST,-122.42158168137,37.7617007179518,"(37.7617007179518, -122.42158168137)",15006027571000,19,1,14,0,0,32
150098210,"ROBBERY, BODILY FORCE",NONE,300 Block of LEAVENWORTH ST,-122.414406029855,37.7841907151119,"(37.7841907151119, -122.414406029855)",15009821003074,1,2,15,6,4,20
150098210,AGGRAVATED ASSAULT WITH BODILY FORCE,NONE,300 Block of LEAVENWORTH ST,-122.414406029855,37.7841907151119,"(37.7841907151119, -122.414406029855)",15009821004014,1,2,15,6,4,13
150098210,DOMESTIC VIOLENCE,NONE,300 Block of LEAVENWORTH ST,-122.414406029855,37.7841907151119,"(37.7841907151119, -122.414406029855)",15009821015200,1,2,15,6,4,3
150098226,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",NONE,LOMBARD ST / LAGUNA ST,-122.431118543788,37.8004687042875,"(37.8004687042875, -122.431118543788)",15009822628160,27,1,19,1,9,28
150098232,AIDED CASE -PROPERTY FOR DESTRUCTION,NONE,400 Block of LOCUST ST,-122.451781767894,37.7870853907529,"(37.7870853907529, -122.451781767894)",15009823251041,1,2,16,6,8,32
150098248,DOMESTIC VIOLENCE,NONE,700 Block of KIRKWOOD AV,-122.374019331833,37.729203356539,"(37.729203356539, -122.374019331833)",15009824815200,31,1,21,5,1,3
150098248,"MALICIOUS MISCHIEF, VANDALISM",NONE,700 Block of KIRKWOOD AV,-122.374019331833,37.729203356539,"(37.729203356539, -122.374019331833)",15009824828150,31,1,21,5,1,28
150098254,"BURGLARY OF STORE, UNLAWFUL ENTRY",NONE,200 Block of STOCKTON ST,-122.40656817787,37.7878092959561,"(37.7878092959561, -122.40656817787)",15009825405053,31,1,16,5,2,37
150098260,PETTY THEFT SHOPLIFTING,NONE,800 Block of GEARY ST,-122.417295322526,37.7862578545865,"(37.7862578545865, -122.417295322526)",15009826006362,31,1,17,5,2,35


#### Input Feature

In [13]:
from pyspark.ml.feature import VectorAssembler

vectorizer = VectorAssembler()
vectorizer.setInputCols(["DayOfWeek", "Month", "Date", "Time", "PdDistrict", "X", "Y"])
vectorizer.setOutputCol("features")


Out[ 12 ]: VectorAssembler_4c07acd1315ccf218e28

#### Training, Test Dataset Splitting

In [15]:
(split20DF, split80DF) = crimeDF.randomSplit([20.0, 80.0])

testSetDF = split20DF.cache()
trainingSetDF = split80DF.cache()

####Classification Evaluator

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

classEval = MulticlassClassificationEvaluator(predictionCol="Prediction_Category", labelCol="Category", metricName="accuracy")

##Random Forest Classifier
- 3-fold crossvalidation 
- Grid-search on "maxBins", "maxDepth", "numTrees" paramters

In [19]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

rf = RandomForestClassifier()
rf.setParams(featuresCol="features", labelCol="Category", predictionCol="Prediction_Category")

rfPipeline = Pipeline()
rfPipeline.setStages([vectorizer, rf])

Out[ 21 ]: Pipeline_4f28a0e4a68d6028d979

In [20]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

crossval = CrossValidator(estimator=rfPipeline, evaluator=classEval, numFolds=3)

paramGrid = (ParamGridBuilder()
            .addGrid(rf.maxBins, [50,100])
            .addGrid(rf.maxDepth, [5, 8])
            .addGrid(rf.numTrees, [20, 25])
            .build())

crossval.setEstimatorParamMaps(paramGrid)
rfModel = crossval.fit(trainingSetDF).bestModel

In [21]:
resultsDF = rfModel.transform(testSetDF)

In [22]:
display(resultsDF)

IncidntNum,Descript,Resolution,Address,X,Y,Location,PdId,Date,Month,Time,DayOfWeek,PdDistrict,Category,features,rawPrediction,probability,Prediction_Category
5012367,"MALICIOUS MISCHIEF, VANDALISM",NONE,1500 Block of JERROLD AV,-122.388476334633,37.7382013082318,"(37.7382013082318, -122.388476334633)",501236728150,22,1,16,5,1,28,"List(1, 7, List(), List(5.0, 1.0, 22.0, 16.0, 1.0, -122.388476334633, 37.7382013082318))","List(1, 39, List(), List(0.21418567386317883, 0.007149120079290514, 0.07276757909650503, 0.5156750217196582, 0.023615851243153983, 1.3632466815366562, 5.53858236318502, 0.048511409710982716, 1.5475884030414804, 0.0926804711327614, 0.2171584096358028, 0.014345751658489989, 0.02208403306322338, 3.1400403373341894, 0.09645038597114759, 0.006505938042786472, 3.508459199622468E-4, 0.5805221794768045, 0.018922566296141318, 1.1675816613275363, 0.6178157421013325, 0.12499676048822231, 0.00655166047256064, 0.027328159142186628, 0.005656513904108716, 0.059294194774110257, 0.06585509716501883, 0.22030514661103123, 1.285533747969993, 1.89233297180919, 6.041818712084748E-4, 0.17848336634624073, 1.7640977111472724, 1.2636691773523467, 0.09169897209094793, 1.7833846676949643, 0.10755634132968901, 0.8081623742271279, 0.008708530167677368))","List(1, 39, List(), List(0.008567426954527151, 2.859648031716205E-4, 0.0029107031638602007, 0.020627000868786324, 9.446340497261592E-4, 0.05452986726146624, 0.22154329452740076, 0.0019404563884393084, 0.061903536121659206, 0.0037072188453104553, 0.008686336385432111, 5.738300663395995E-4, 8.833613225289351E-4, 0.12560161349336757, 0.003858015438845903, 2.6023752171145883E-4, 1.403383679848987E-5, 0.023220887179072176, 7.569026518456526E-4, 0.046703266453101445, 0.024712629684053296, 0.004999870419528892, 2.6206641890242553E-4, 0.001093126365687465, 2.2626055616434862E-4, 0.0023717677909644097, 0.0026342038866007528, 0.008812205864441248, 0.051421349918799715, 0.07569331887236759, 2.4167274848338988E-5, 0.007139334653849629, 0.07056390844589089, 0.050546767094093864, 0.003667958883637917, 0.07133538670779856, 0.00430225365318756, 0.03232649496908511, 3.4834120670709464E-4))",6.0
11473002,WARRANT ARREST,"ARREST, CITED",800 Block of BRYANT ST,-122.403404791479,37.775420706711,"(37.775420706711, -122.403404791479)",1147300263010,17,4,10,1,7,8,"List(1, 7, List(), List(1.0, 4.0, 17.0, 10.0, 7.0, -122.403404791479, 37.775420706711))","List(1, 39, List(), List(0.7596164999352197, 0.017499158986130817, 0.027541175563367326, 0.27583878749980756, 0.008441087791159974, 0.6096981614061925, 3.1011610459313537, 0.02567876328547734, 1.8383144196613177, 0.024492540629741034, 0.47746540324019987, 0.0030206748217984107, 0.014225250072735354, 1.6728447098449328, 0.07094362795595129, 0.011396360486217103, 0.0, 0.1600468238708106, 0.033861493171272176, 1.0358995229347392, 0.3538169699441474, 0.19475197293444324, 0.026350547287131255, 0.03462906248474435, 0.015030249389649666, 0.06339619176164338, 0.021871732443457186, 0.040204322949818086, 0.9429654400050141, 0.738975321906532, 0.0, 0.17598991284322105, 4.675786776099074, 0.4098528606209792, 0.11233325934087424, 6.472042873587364, 0.05195907073234956, 0.490489308291396, 0.011568620289736032))","List(1, 39, List(), List(0.030384659997408785, 6.999663594452326E-4, 0.001101647022534693, 0.0110335514999923, 3.376435116463989E-4, 0.024387926456247697, 0.12404644183725413, 0.0010271505314190935, 0.0735325767864527, 9.797016251896413E-4, 0.019098616129607993, 1.2082699287193641E-4, 5.690100029094141E-4, 0.0669137883937973, 0.002837745118238051, 4.55854419448684E-4, 0.0, 0.006401872954832423, 0.001354459726850887, 0.041435980917389564, 0.014152678797765895, 0.007790078917377728, 0.00105402189148525, 0.0013851624993897738, 6.012099755859866E-4, 0.002535847670465735, 8.748692977382873E-4, 0.0016081729179927232, 0.03771861760020056, 0.029559012876261278, 0.0, 0.007039596513728841, 0.18703147104396295, 0.016394114424839165, 0.004493330373634969, 0.25888171494349455, 0.00207

In [23]:
evalRF = classEval.evaluate(resultsDF)

In [24]:
print("RF Accuracy: {0:.2f}".format(evalRF))

RF Accuracy: 0.26

##Multilayer Perceptron Classifier
- consists of 4 layers of size [7, 50, 39]
  - input feature consists of 7 elements
  - output is 1 of 39 different categories

In [26]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline

layers = [7, 50, 39]

mlp = MultilayerPerceptronClassifier().setLayers(layers).setLabelCol("Category").setFeaturesCol("features").setPredictionCol("Prediction_Category")

mlpPipeline = Pipeline()
mlpPipeline.setStages([vectorizer, mlp])

mlpModel = mlpPipeline.fit(trainingSetDF)

In [27]:
resultsMLP = mlpModel.transform(testSetDF)

In [28]:
display(resultsMLP)

IncidntNum,Descript,Resolution,Address,X,Y,Location,PdId,Date,Month,Time,DayOfWeek,PdDistrict,Category,features,rawPrediction,probability,Prediction_Category
4129175,"DEATH REPORT, CAUSE UNKNOWN",NONE,600 Block of 39TH AV,-122.499452303575,37.7765353836561,"(37.7765353836561, -122.499452303575)",412917561030,11,11,19,3,8,32,"List(1, 7, List(), List(3.0, 11.0, 11.0, 19.0, 8.0, -122.499452303575, 37.7765353836561))","List(1, 39, List(), List(1.3981977505616574, -1.7877863937496439, -0.5909965075565105, 1.0229334190576276, -1.8696860135590256, 1.9388172090330573, 3.3754229713058854, -0.3063467062343229, 2.343994611088526, -0.8974620695550541, 0.7149980878583433, -3.2869582026222948, -2.0662178265857776, 3.0991787568898874, 0.5304955811535542, -2.256467885094541, -8.836185540978985, 1.3455864735780372, -0.9966634050772212, 1.9718647608378863, 1.970296745533694, 0.3862454190279898, 0.6587124077915715, -1.250911800594269, -2.123663146452565, 0.2912870976755596, -0.65281047545095, 0.08195318112924191, 2.9190584130930364, 2.5428396994245386, -7.609163980968226, 0.7646870735677197, 3.3479004947625457, 3.0845020438597808, 0.4629060729281958, 4.633584010836828, -0.4550432808525492, 2.4248416600120475, -2.9549604796410978))","List(1, 39, List(), List(0.013180329527677592, 5.448424474014295E-4, 0.0018031430349992532, 0.009056304992170283, 5.019984568701886E-4, 0.022631550879973535, 0.09519722113434542, 0.0023969115969469153, 0.0339375608735616, 0.001327192369751346, 0.0066560476880117535, 1.2167150809203665E-4, 4.124294796339354E-4, 0.0722192432351904, 0.005534622499820984, 3.409770504166314E-4, 4.7335839006607813E-7, 0.012504821049085435, 0.0012018528110227162, 0.023391963919520323, 0.02335531370365832, 0.004791163095253363, 0.006291755893354875, 9.320358304989682E-4, 3.8940499209778726E-4, 0.004357135639182806, 0.0016950585870209066, 0.003534178780859606, 0.060315323553195105, 0.04140358534942798, 1.6146513940772503E-6, 0.006995134625319532, 0.09261288465013764, 0.0711670424389982, 0.005172902035724136, 0.3349937219283101, 0.002065731703420151, 0.03679527500441489, 1.6957962484965593E-4))",35.0
15079313,"DRIVERS LICENSE, SUSPENDED OR REVOKED","ARREST, BOOKED",MASONIC AV / OAK ST,-122.445717668922,37.7720809772392,"(37.7720809772392, -122.445717668922)",1507931365016,9,9,22,2,6,6,"List(1, 7, List(), List(2.0, 9.0, 9.0, 22.0, 6.0, -122.445717668922, 37.7720809772392))","List(1, 39, List(), List(1.2341883350699252, -1.7481245388937858, -0.5326744068846705, 1.11207278720383, -2.0291827934120064, 1.992643541235101, 3.5500570350234817, -0.12499484199967267, 2.45671358258885, -0.7938414105849925, 0.6155718682168734, -3.232034765731486, -2.1608273394312265, 3.14441886169806, 0.39688648104839896, -2.3659168798195798, -8.682503550836504, 1.2468446458776525, -1.1561945419128365, 2.0278465510924013, 2.0819285950133146, 0.33599797846570545, 0.8967669144116627, -1.1739909129667312, -2.33827633771157, 0.34989529057786406, -0.8254318772380875, -0.04873626516853308, 2.9044647322629515, 2.768124020715447, -7.798507363727661, 0.5911634971294634, 3.3004068184017434, 3.153139123858264, 0.39204307553965523, 4.34083242277224, -0.47519540032068874, 2.328206116284397, -2.9304629582776784))","List(1, 39, List(), List(0.011785879753140384, 5.972552574116274E-4, 0.002013832135254802, 0.010431046880015483, 4.5091837406253607E-4, 0.025162565398590755, 0.11943483201106385, 0.0030274449312823496, 0.04002192509676457, 0.0015509574318348215, 0.00634892633174741, 1.3542721834520447E-4, 3.9529874216748473E-4, 0.07960944252296062, 0.005101835259620638, 3.2200022617139386E-4, 5.815639010255505E-7, 0.011935993449447239, 0.0010795230323407715, 0.026064139390904063, 0.027512554940810823, 0.004800460386857016, 0.008410498636987758, 0.0010604813786190635, 3.3102463217250165E-4, 0.004867639607211913, 0.0015027277723910686, 0.0032673445265485974, 0.06262587811105819, 0.054643915327408615, 1.4077064714650645E-6, 0.006195835333277636, 0.09304848181702693, 0.08030669341907772, 0.00507

In [29]:
evalMLP = classEval.evaluate(resultsMLP)

In [30]:
print("MLP Accuracy: {0:.2f}".format(evalMLP))

MLP Accuracy: 0.22